# 🔄 Churn Voorspelling met een Neural Network

In dit notebook bouwen we stap voor stap een **neural network** dat voorspelt of een klant gaat opzeggen ("churnen").

We gebruiken een dataset van een telecombedrijf met klantgegevens zoals belminuten, kosten en klantenservice-interacties.

### Wat gaan we doen?
1. Data laden en verkennen
2. Data voorbereiden voor het model
3. Een neural network trainen
4. Resultaten bekijken en evalueren

---
**Instructie:** Voer iedere cel uit met **Shift+Enter**

## Stap 1: Data laden

We laden de dataset in en bekijken de eerste rijen. Elke rij is één klant.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Laad de dataset
df = pd.read_csv('data/churn-bigml-20.csv')

print(f"De dataset bevat {len(df)} klanten en {len(df.columns)} kolommen")
print(f"Churn-percentage: {df['Churn'].mean():.1%} van de klanten is vertrokken")
print()
df.head()

### Wat zit er in de data?

De belangrijkste kolommen:
- **Account length**: hoe lang de klant al klant is
- **International plan / Voice mail plan**: wel of geen abonnement
- **Total day/eve/night minutes**: belminuten per dagdeel
- **Customer service calls**: hoe vaak de klant de klantenservice heeft gebeld
- **Churn**: ons **doel** — True = klant is vertrokken

In [ ]:
# Laten we de verdeling van churn visualiseren
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Churn verdeling
df['Churn'].value_counts().plot(kind='bar', ax=axes[0], color=['steelblue', 'coral'])
axes[0].set_title('Churn Verdeling')
axes[0].set_xticklabels(['Gebleven', 'Vertrokken'], rotation=0)
axes[0].set_ylabel('Aantal klanten')

# Klantenservice calls vs churn
df.groupby('Customer service calls')['Churn'].mean().plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_title('Churn-percentage per aantal klantenservice calls')
axes[1].set_ylabel('% vertrokken')
axes[1].set_xlabel('Aantal klantenservice calls')

plt.tight_layout()
plt.show()

## Stap 2: Data voorbereiden

Een neural network kan alleen met **getallen** werken. We moeten dus:
1. Tekst-kolommen omzetten naar getallen ("one-hot encoding")
2. Alle getallen op dezelfde schaal brengen ("feature scaling")
3. De data splitsen in een **train-set** (om van te leren) en een **test-set** (om te evalueren)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Stap 2a: Churn omzetten van True/False naar 1/0
df['Churn'] = df['Churn'].astype(int)

# Stap 2b: Tekst-kolommen omzetten met one-hot encoding
# Bijvoorbeeld: 'International plan' = 'Yes'/'No' wordt twee kolommen met 0 en 1
categorical_cols = df.select_dtypes(include='object').columns
print(f"Tekst-kolommen die we omzetten: {list(categorical_cols)}")

df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Stap 2c: Features (X) en doel (y) scheiden
X = df_encoded.drop('Churn', axis=1)
y = df_encoded['Churn']

# Stap 2d: Feature scaling — alle getallen op dezelfde schaal brengen
# Dit is belangrijk omdat een neural network anders te veel focust op kolommen met grote getallen
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

print(f"\nVoor scaling — gemiddelde 'Total day minutes': {X['Total day minutes'].mean():.1f}")
print(f"Na scaling  — gemiddelde 'Total day minutes': {X_scaled['Total day minutes'].mean():.4f}")
print("(Na scaling is het gemiddelde ~0 en de standaardafwijking ~1)")

# Stap 2e: Splitsen in train en test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTrain-set: {len(X_train)} klanten")
print(f"Test-set:  {len(X_test)} klanten")

## Stap 3: Neural Network trainen

We gebruiken een **MLPClassifier** (Multi-Layer Perceptron) — een neural network met:
- Een **input laag** die alle klantgegevens ontvangt
- Een **hidden laag** met 64 neuronen die patronen leert herkennen
- Een **output laag** die voorspelt: "churn" of "geen churn"

Het model leert door steeds opnieuw de data te bekijken en zijn gewichten aan te passen (backpropagation + gradient descent).

In [ ]:
from sklearn.neural_network import MLPClassifier

# Maak het neural network aan
model = MLPClassifier(
    hidden_layer_sizes=(64,),    # 1 hidden laag met 64 neuronen
    max_iter=500,                # maximaal 500 keer door de data
    random_state=42,             # voor reproduceerbaarheid
    verbose=True                 # toon voortgang tijdens het trainen
)

# Train het model
print("Model wordt getraind...\n")
model.fit(X_train, y_train)
print("\n✅ Training voltooid!")

### Training visualiseren

Hieronder zien we hoe de **loss** (fout) afneemt tijdens het trainen. Dit is gradient descent in actie!

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(model.loss_curve_, color='steelblue', linewidth=2)
plt.title('Training Loss per Iteratie (Gradient Descent in actie)')
plt.xlabel('Iteratie')
plt.ylabel('Loss (hoe lager, hoe beter)')
plt.grid(True, alpha=0.3)
plt.show()

## Stap 4: Resultaten evalueren

We testen het model op data die het **nog niet eerder heeft gezien** (de test-set).

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Voorspellingen maken op de test-set
y_pred = model.predict(X_test)

# Resultaten berekenen
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary', zero_division=0)
recall = recall_score(y_test, y_pred, average='binary', zero_division=0)
f1 = f1_score(y_test, y_pred, average='binary', zero_division=0)

print("📊 Model Resultaten op de test-set:")
print(f"   Accuracy:  {accuracy:.1%}  — van alle klanten, hoeveel correct voorspeld?")
print(f"   Precision: {precision:.1%}  — als we 'churn' zeggen, hoe vaak klopt dat?")
print(f"   Recall:    {recall:.1%}  — van alle echte churners, hoeveel hebben we gevonden?")
print(f"   F1-Score:  {f1:.1%}  — balans tussen precision en recall")

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

fig, ax = plt.subplots(figsize=(6, 5))
ConfusionMatrixDisplay.from_estimator(
    model, X_test, y_test, 
    display_labels=['Gebleven', 'Vertrokken'],
    cmap=plt.cm.Blues, ax=ax
)
ax.set_title('Confusion Matrix')
plt.show()

print("Hoe lees je dit?")
print("- Linksboven: correct voorspeld als 'gebleven'")
print("- Rechtsonder: correct voorspeld als 'vertrokken'")
print("- Rechtsboven: fout! We zeiden 'vertrokken' maar klant bleef")
print("- Linksonder: fout! We zeiden 'gebleven' maar klant vertrok")

## Stap 5: Voorspelling voor één klant

Laten we het model gebruiken om voor één specifieke klant te voorspellen.

In [ ]:
# Pak een klant uit de test-set
klant = X_test.iloc[0:1]
echte_waarde = y_test.iloc[0]

# Voorspelling
voorspelling = model.predict(klant)[0]
kans = model.predict_proba(klant)[0]

print("🔍 Voorspelling voor één klant:")
print(f"   Kans op blijven:    {kans[0]:.1%}")
print(f"   Kans op vertrekken: {kans[1]:.1%}")
print(f"   Voorspelling: {'Vertrokken' if voorspelling == 1 else 'Gebleven'}")
print(f"   Werkelijkheid: {'Vertrokken' if echte_waarde == 1 else 'Gebleven'}")
print(f"   {'✅ Correct!' if voorspelling == echte_waarde else '❌ Fout!'}")

## 🧪 Experimenteer zelf!

Probeer het model te verbeteren door de instellingen aan te passen:

```python
# Meer neuronen:
model = MLPClassifier(hidden_layer_sizes=(128,), max_iter=500, random_state=42)

# Twee hidden lagen:
model = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)

# Andere learning rate:
model = MLPClassifier(hidden_layer_sizes=(64,), max_iter=500, learning_rate_init=0.01, random_state=42)
```

Kopieer één van bovenstaande regels, plak het in de cel hieronder, en train opnieuw!

In [ ]:
# Experimenteer hier!
# Pas het model aan en kijk of je betere resultaten krijgt

model2 = MLPClassifier(hidden_layer_sizes=(64,), max_iter=500, random_state=42)
model2.fit(X_train, y_train)

y_pred2 = model2.predict(X_test)
print(f"Accuracy:  {accuracy_score(y_test, y_pred2):.1%}")
print(f"F1-Score:  {f1_score(y_test, y_pred2, average='binary', zero_division=0):.1%}")